In [1]:
# Data Processing from base data to dataset for inflation adjustment factor analysis

In [2]:
import pandas as pd

df_clean = pd.read_json("../../public/data/base_data.json")
df_inf = pd.read_excel("../../public/data/Trimmed Mean PCE Inflation.xlsx")

entities = df_clean['Entity Name'].unique().tolist()
years = df_clean["Tax Year"].unique().tolist()

rate_growth_cap = 10

# merge inflation variable 
df_clean = df_clean.merge(df_inf, how='left', on="Tax Year")

df_clean['Inflation'] = (df_clean['inflation_rate_rolling']/100) + 1


# drop needless columns
df_clean = df_clean[['Tax Year', 'County Name', 'Entity Name', 'Entity Code', 'County Code', 'Entity Type', 'Tax Rate' , 
                 'Total Taxable Property', 'Revenue, Total', 'Inflation']]


In [3]:
# Calculate inflated Revenue

df_clean["Inflated Revenue"] = df_clean["Revenue, Total"]

for year in years:
    for entity in entities:
        if year == 2015:
            alpha = df_clean.loc[(df_clean["Tax Year"] == year) & (df_clean['Entity Name']== entity)]["Revenue, Total"].sum() * df_clean.loc[(df_clean["Tax Year"] == year) & (df_clean['Entity Name']== entity)]["Inflation"].sum()
            df_clean.loc[(df_clean["Tax Year"] == year) & (df_clean['Entity Name']== entity),"Inflated Revenue"] = alpha
        
        if year >= 2016:
            alpha = df_clean.loc[(df_clean["Tax Year"] == year - 1) & (df_clean['Entity Name']== entity)]["Inflated Revenue"].sum() * df_clean.loc[(df_clean["Tax Year"] == year) & (df_clean['Entity Name']== entity)]["Inflation"].sum()
            df_clean.loc[(df_clean["Tax Year"] == year) & (df_clean['Entity Name']== entity),"Inflated Revenue"] = alpha

# Calculate inflation differential
df_clean['Revenue Differential'] = df_clean['Inflated Revenue'] - df_clean['Revenue, Total']


In [4]:
# Compute Observed Growth Rates
for year in years:
    for entity in entities:
        if year == 2014:
            df_clean['Observed Growth Rate'] = 0
        
        if year >= 2015:
            df_clean.loc[(df_clean["Tax Year"] == year) & (df_clean['Entity Name']== entity),'Observed Growth Rate'] = ((df_clean.loc[(df_clean["Tax Year"] == year) & (df_clean['Entity Name']== entity)]["Revenue, Total"].sum()/df_clean.loc[(df_clean["Tax Year"] == year -1) & (df_clean['Entity Name']== entity)]["Revenue, Total"].sum())-1)*100

df_clean['Inflation Rate'] = (df_clean['Inflation']-1)*100

C:\Users\jarhe\AppData\Local\Temp\ipykernel_26060\3596349781.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '8.872303002135574' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_clean.loc[(df_clean["Tax Year"] == year) & (df_clean['Entity Name']== entity),'Observed Growth Rate'] = ((df_clean.loc[(df_clean["Tax Year"] == year) & (df_clean['Entity Name']== entity)]["Revenue, Total"].sum()/df_clean.loc[(df_clean["Tax Year"] == year -1) & (df_clean['Entity Name']== entity)]["Revenue, Total"].sum())-1)*100
C:\Users\jarhe\AppData\Local\Temp\ipykernel_26060\3596349781.py:8: RuntimeWarning: invalid value encountered in scalar divide
  df_clean.loc[(df_clean["Tax Year"] == year) & (df_clean['Entity Name']== entity),'Observed Growth Rate'] = ((df_clean.loc[(df_clean["Tax Year"] == year) & (df_clean['Entity Name']== entity)]["Revenue, Total"].sum()/df_clean.loc[(df_clea

In [5]:
# Capped growth scenario

# compounded 10% growth of total revenue
df_clean['10 Percent Revenue'] = df_clean['Revenue, Total']

for year in years:
    for entity in entities:
        
        if year >= 2015:
            delta = df_clean.loc[(df_clean["Tax Year"] == year- 1 ) & (df_clean['Entity Name']== entity)]["10 Percent Revenue"].sum() * ((rate_growth_cap/100)+1)
            df_clean.loc[(df_clean["Tax Year"] == year) & (df_clean['Entity Name']== entity),"10 Percent Revenue"] = delta

# Capped growth rate
df_clean["Capped Growth Rate"] = df_clean['Observed Growth Rate']

for year in years:
    for entity in entities:
        if df_clean.loc[(df_clean["Tax Year"] == year ) & (df_clean['Entity Name']== entity)]["Observed Growth Rate"].sum() > rate_growth_cap:
            df_clean.loc[(df_clean["Tax Year"] == year) & (df_clean['Entity Name']== entity),"Capped Growth Rate"] = rate_growth_cap

# Capped growth total revenue
df_clean['Capped Revenue'] = df_clean['Revenue, Total']

for year in years:
    for entity in entities:
        if year == 2015:
            beta = df_clean.loc[(df_clean["Tax Year"] == year- 1 ) & (df_clean['Entity Name']== entity)]["Revenue, Total"].sum() * ((df_clean.loc[(df_clean["Tax Year"] == year) & (df_clean['Entity Name']== entity)]["Capped Growth Rate"].sum()/100)+1)
            df_clean.loc[(df_clean["Tax Year"] == year) & (df_clean['Entity Name']== entity),"Capped Revenue"] = beta
        
        if year >= 2016:
            beta = df_clean.loc[(df_clean["Tax Year"] == year- 1 ) & (df_clean['Entity Name']== entity)]["Capped Revenue"].sum() * ((df_clean.loc[(df_clean["Tax Year"] == year) & (df_clean['Entity Name']== entity)]["Capped Growth Rate"].sum()/100)+1)
            df_clean.loc[(df_clean["Tax Year"] == year) & (df_clean['Entity Name']== entity),"Capped Revenue"] = beta



In [6]:
# Compute Tax Rates
# Capped growth tax rates
for year in years:
    for entity in entities:
        gamma = df_clean.loc[(df_clean["Tax Year"] == year) & (df_clean["Entity Name"]== entity)]['Capped Revenue'].sum()/ (df_clean.loc[(df_clean["Tax Year"] == year) & (df_clean["Entity Name"]== entity)]['Total Taxable Property'].sum()*1000)
        df_clean.loc[(df_clean["Tax Year"] == year) & (df_clean['Entity Name']== entity),"Capped Tax Rate"] = gamma


df_clean['Inflated Tax Rate'] = df_clean['Inflated Revenue']/(df_clean['Total Taxable Property'] * 1000)

C:\Users\jarhe\AppData\Local\Temp\ipykernel_26060\1606898624.py:5: RuntimeWarning: invalid value encountered in scalar divide
  gamma = df_clean.loc[(df_clean["Tax Year"] == year) & (df_clean["Entity Name"]== entity)]['Capped Revenue'].sum()/ (df_clean.loc[(df_clean["Tax Year"] == year) & (df_clean["Entity Name"]== entity)]['Total Taxable Property'].sum()*1000)
C:\Users\jarhe\AppData\Local\Temp\ipykernel_26060\1606898624.py:5: RuntimeWarning: invalid value encountered in scalar divide
  gamma = df_clean.loc[(df_clean["Tax Year"] == year) & (df_clean["Entity Name"]== entity)]['Capped Revenue'].sum()/ (df_clean.loc[(df_clean["Tax Year"] == year) & (df_clean["Entity Name"]== entity)]['Total Taxable Property'].sum()*1000)
C:\Users\jarhe\AppData\Local\Temp\ipykernel_26060\1606898624.py:5: RuntimeWarning: invalid value encountered in scalar divide
  gamma = df_clean.loc[(df_clean["Tax Year"] == year) & (df_clean["Entity Name"]== entity)]['Capped Revenue'].sum()/ (df_clean.loc[(df_clean["Tax 

In [7]:
# Export to JSON
df_clean.to_json('../../public/data/inflation_data.json', orient = 'records')